Import related packages

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import operator
nltk.download('punkt') #If needed
nltk.download('stopwords') #If needed
nltk.download('wordnet') #If needed

from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Import dataset    
The original data has been divided into training set, test set and validation set

In [ ]:
#Import training set data and label
df1 = pd.read_table('train_text.txt', header = None)
df2 = pd.read_table('train_labels.txt', header = None)
train_data = pd.concat([df1, df2], axis = 1)
train_data.columns =['0', '1']

#Import testing set data and label
df3 = pd.read_table('test_text.txt', header = None)
df4 = pd.read_table('test_labels.txt', header = None)
test_data = pd.concat([df3, df4], axis = 1)
test_data.columns =['0', '1']

#Import validation set data and label
df5 = pd.read_table('val_text.txt', header = None)
df6 = pd.read_table('val_labels.txt', header = None)
val_data = pd.concat([df5, df6], axis = 1)
val_data.columns =['0', '1']

In [ ]:
train_data.head()

,0,1
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [ ]:
train_data.to_csv(r"./result/train_with_labels.csv",header =True, index = False, encoding = "utf_8_sig")
val_data.to_csv(r"./result/val_with_labels.csv",header =True, index = False, encoding = "utf_8_sig")
test_data.to_csv(r"./result/test_with_labels.csv",header =True, index = False, encoding = "utf_8_sig")

In [ ]:
# !!!after installing v0.9 torchtext, if you using google-colab, pleas use Runtime -> Restart runtime!!!
!pip install torchtext==0.9

In [ ]:
import torch
import torchtext
print(torchtext.__version__)    # double check the torch version

from torchtext.legacy import data

0.9.0


In [ ]:
# 添加cleanup_text 最终决定不用clean stopwords
# import re
# pattern1 = ['.','@','#','!','&',"'",'?',',',"n't","'s",'...',"'m",';','-',':','u','(',')','/']
# pattern2 = ['@user','+%', 'ahahah', 'am', '..', '...', 'He', 'he', 'She', 'she', 'it', 'It', 'Am', 'am', 'The', 'the', 'My','Im','.','#',',',"'s","'m",';','-','(',')','/']

# def cleanup_text(texts):
#   cleaned_text = []
#   for text in texts:
#     if text not in pattern2 or text not in pattern1:
#       text = re.sub('[^a-zA-Z0-9]', ' ', text)
#       text = re.sub(r' +', ' ', text)
#       text = re.sub(r'\n', ' ', text)
#       cleaned_text.append(text)
#   return cleaned_text


In [ ]:
# 定义Field，一个能够加载，预处理和存储文本数据和标签的对象
# 之后用它根据训练数据来建立词表，加载预训练Glove词向量
import random

def DataLoader():
  SEED = 1234

  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

  # TEXT = data.Field(preprocessing=cleanup_text,
  TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language = 'en_core_web_sm', include_lengths = False)
                    # include_lengths = True)

  # LABEL = data.LabelField(dtype = torch.float)
  LABEL = data.LabelField(dtype = torch.long)
  fields = [('text', TEXT), ('label', LABEL)]

  train_data, valid_data, test_data = data.TabularDataset.splits(path = './result',
                                                                 train = 'train_with_labels.csv',
                                                                 validation = 'val_with_labels.csv',
                                                                 test = 'test_with_labels.csv',
                                                                 format = 'csv',
                                                                 fields = fields,
                                                                 skip_header = True)


  return TEXT, LABEL, train_data, valid_data, test_data

In [ ]:
# 加载完数据之后，开始建词表
TEXT, LABEL, train_data, valid_data, test_data = DataLoader()

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                #  vectors = "glove.6B.100d", 
                 vectors = "glove.twitter.27B.100d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.twitter.27B.zip: 1.52GB [04:45, 5.32MB/s]                            
100%|█████████▉| 1193513/1193514 [00:49<00:00, 24049.85it/s]


In [ ]:
print(TEXT.vocab.freqs.most_common(20)) # 训练集中最常出现的词
print(TEXT.vocab.itos[:10]) # 列表 index to word
print(TEXT.vocab.stoi)  # 字典 word to index
print(LABEL.vocab.stoi)

[('#', 3239), ('@user', 2019), ('.', 1909), ('the', 1373), ('I', 1359), ('to', 1204), (',', 1079), ('a', 1004), ('and', 846), ('!', 763), ('is', 739), (' ', 722), ('of', 671), ('you', 638), ('in', 552), ('it', 527), ("'s", 477), ("n't", 476), ('that', 473), ('my', 435)]
['<unk>', '<pad>', '#', '@user', '.', 'the', 'I', 'to', ',', 'a']
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f08b1502a90>>, {'<unk>': 0, '<pad>': 1, '#': 2, '@user': 3, '.': 4, 'the': 5, 'I': 6, 'to': 7, ',': 8, 'a': 9, 'and': 10, '!': 11, 'is': 12, ' ': 13, 'of': 14, 'you': 15, 'in': 16, 'it': 17, "'s": 18, "n't": 19, 'that': 20, 'my': 21, '?': 22, 'for': 23, "'": 24, 'on': 25, 'me': 26, 'do': 27, 'with': 28, 'have': 29, 'be': 30, 'so': 31, 'not': 32, "'m": 33, 'was': 34, 'just': 35, 'this': 36, 'are': 37, 'at': 38, 'but': 39, 'i': 40, 'your': 41, 'like': 42, ';': 43, '...': 44, '-': 45, 'all': 46, 'about': 47, '&': 48, 'out': 49, 'up': 50, 'get': 51, 'by': 52, 'he': 53, '

In [ ]:
# sort_within_batch 设置为True的话，每一个batch内的数据，会根据sort_key的排列规则降序排列
# len(x.text)即text的长度，即每条用户所包含的单词数量
# TODO: Python里难道没有办法类似于继承Comparable<T>接口，然后重写CompareTo()来排序吗，要这样在方法里声明?

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    device = device)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,079,904 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0188,  0.0117,  0.0393,  ...,  0.4428,  0.9039, -0.2370],
        ...,
        [-1.0419,  1.3155,  0.8454,  ..., -0.6409, -0.5735,  0.6514],
        [-0.9408, -0.6966, -0.3996,  ...,  0.8543, -1.3833, -2.2067],
        [-1.6231, -0.3696,  0.6649,  ..., -0.6217,  0.9142, -0.2956]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0188,  0.0117,  0.0393,  ...,  0.4428,  0.9039, -0.2370],
        ...,
        [-1.0419,  1.3155,  0.8454,  ..., -0.6409, -0.5735,  0.6514],
        [-0.9408, -0.6966, -0.3996,  ...,  0.8543, -1.3833, -2.2067],
        [-1.6231, -0.3696,  0.6649,  ..., -0.6217,  0.9142, -0.2956]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 1.244 | Train Acc: 44.48%
	 Val. Loss: 1.174 |  Val. Acc: 53.34%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 1.091 | Train Acc: 54.32%
	 Val. Loss: 1.089 |  Val. Acc: 54.48%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.904 | Train Acc: 66.31%
	 Val. Loss: 0.978 |  Val. Acc: 63.00%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.727 | Train Acc: 73.53%
	 Val. Loss: 0.868 |  Val. Acc: 66.96%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.557 | Train Acc: 80.67%
	 Val. Loss: 0.809 |  Val. Acc: 68.78%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.423 | Train Acc: 86.15%
	 Val. Loss: 0.817 |  Val. Acc: 69.44%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.344 | Train Acc: 88.83%
	 Val. Loss: 0.787 |  Val. Acc: 69.75%
Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.274 | Train Acc: 91.55%
	 Val. Loss: 0.803 |  Val. Acc: 70.06%
Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 0.207 | Train Acc: 94.34%
	 Val. Loss: 0.814 |  Val. Acc: 71.53%
Epoch: 10 | Epoch Time: 0m 3

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.727 | Test Acc: 73.24%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [ ]:
# map Integer label -> String emotions
def labelToemotion(label):
  if (label == '0'):
      return 'anger'
  elif (label == '1'):
    return 'joy'
  elif (label == '2'):
    return 'optimism'
  else:
    return 'sadness'
  

In [ ]:
pred_class = predict_class(model, "I am coding test cases")
print('Predicted class is: ' + labelToemotion(LABEL.vocab.itos[pred_class]))

Predicted class is: sadness


In [ ]:
pred_class = predict_class(model, "You are dog")
print('Predicted class is: ' + labelToemotion(LABEL.vocab.itos[pred_class]))

Predicted class is: anger


In [ ]:
pred_class = predict_class(model, "I will finish the code soon.")
print('Predicted class is: ' + labelToemotion(LABEL.vocab.itos[pred_class]))

Predicted class is: joy


In [ ]:
pred_class = predict_class(model, "JAVA is the best language in the world!")
print('Predicted class is: ' + labelToemotion(LABEL.vocab.itos[pred_class]))

Predicted class is: joy


In [ ]:
pred_class = predict_class(model, "All finished!")
print('Predicted class is: ' + labelToemotion(LABEL.vocab.itos[pred_class]))

Predicted class is: joy
